In [4]:
import numpy as np
import cvxpy as cp
from basis_generator import rand_moment, sel_seq, proj_mul
import json

In [35]:
def single_behavior_visibility(X, X_basis):
    
    eta = cp.Variable((1, 1))
    alpha = cp.Variable((len(X_basis), 1))
    beta = cp.Variable((len(X_basis), 1))
    M = cp.Variable(X_basis[0].shape)
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1 - eta]

    for i in range(1,len(X)):
        constraints += [
            eta*X[i,i] + N[i,i] == M[i,i]
        ]

    constraints += [M >> 0]
    constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [M[0,0] == 1]

    prob = cp.Problem(cp.Maximize(eta),
                      constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    coef = []
    for __ in range(3,len(constraints)-3):
        coef.append(np.real(constraints[__].dual_value[0][0]))
    
    return eta.value[0][0], coef

In [36]:
def NPA_bound(coef,X_basis):
    beta = cp.Variable((len(X_basis), 1))
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1]

    prob = cp.Problem(cp.Minimize(
            sum([N[j+1,j+1]*coef[j] for j in range(len(coef))])
            ),constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    return prob.value

In [40]:
# Import basis
basis_filename = "data_basis/2-dim-3-num_obs-2-len_seq-2-num_out-1-level.npy"
X_basis = np.load(basis_filename)

In [41]:
dimX = 3
dim_base = 2
num_obs = 2
len_seq = 2
num_out = 2
remove_last_out = True

In [42]:
X, rho, P = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
eta, coef = single_behavior_visibility(X, X_basis)

In [43]:
for __ in range(5):
    X_t, rho_t, P_t = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
    eta_t, coef_t = single_behavior_visibility(X_t, X_basis)
    if eta_t < eta:
        eta = eta_t
        coef = coef_t
        X = X_t
        rho = rho_t
        P = P_t

print(eta)

0.7421789215215785


In [44]:
Q = np.real(np.diag(X)[1:] @ coef)
print(Q)

-0.9085842392451938


In [45]:
C = NPA_bound(coef,X_basis)
print(C)

-0.6507631284230208


In [46]:
data = {}
# data["Moment Matrix"] = X
# data["Quantum State"] = rho
# data["Measurements"] = P
data["Inequality"] = coef
data["Classical Bound"] = C
data["Quantum Bound"] = Q

data["num of observables"] = num_obs
data["maximum length of sequences"] = len_seq
data["num of outcomes"] = num_out
data["dimension behavior"] = dimX
data["dimension base"] = dim_base

NAME = '{}-num_obs-{}-len_seq-{}-out_max-{}-dim_behavior-{}-dim_base'.format(num_obs, len_seq, num_out, dimX, dim_base)

with open("dimension_witness/" + NAME + '.json', 'w') as fp:
    json.dump(data, fp, indent=2)
    
np.save("dimension_witness/" + NAME + "-moment_matrix", X)
np.save("dimension_witness/" + NAME + "-state", rho)
np.save("dimension_witness/" + NAME + "-measurements", P)

In [47]:
"dimension_witness/" + NAME + "-state"

'dimension_witness/2-num_obs-2-len_seq-2-out_max-3-dim_behavior-2-dim_base-state'

## Verification

In [63]:
dimX = 3
dim_base = 2
num_obs = 3
len_seq = 2
num_out = 2
remove_last_out = True

In [64]:
NAME = '{}-num_obs-{}-len_seq-{}-out_max-{}-dim_behavior-{}-dim_base'.format(num_obs, len_seq, num_out, dimX, dim_base)

In [65]:
#### Loading and testing state
rho = np.load("dimension_witness/" + NAME + "-state.npy")
print(rho.shape)
np.trace(rho @ rho)

(3, 3)


(0.9999999999999997+0j)

In [66]:
#### Loading measurements
P = np.load("dimension_witness/" + NAME + "-measurements.npy")
print(P.shape)
np.trace(rho @ rho)

(3, 2, 3, 3)


(0.9999999999999997+0j)

In [67]:
sequences = sel_seq(num_obs, len_seq, num_out, remove_last_out=True)

In [68]:
X = np.eye(len(sequences)+1, dtype=complex)
for i, seq_row in enumerate(sequences):
    Pi = proj_mul([P[k] for k in seq_row[1]], seq_row[0])
    for j, seq_col in enumerate(sequences):
        if i==j:
            Pj= proj_mul([P[k] for k in seq_col[1]], seq_col[0])
            X[i+1,j+1] = np.trace(Pi @ np.conjugate(Pj.T) @ rho)

In [69]:
with open("dimension_witness/" + NAME + '.json') as json_file:
     ineq = json.load(json_file)

In [70]:
np.real(np.diag(X)[1:] @ ineq["Inequality"])

-0.6372680915558379

In [71]:
NPA_bound(ineq["Inequality"],X_basis)

-0.4790466314231945

In [62]:
eta, coef = single_behavior_visibility(X, X_basis)

In [79]:
eta = cp.Variable((1, 1))
alpha = cp.Variable((len(X_basis), 1))
beta = cp.Variable((len(X_basis), 1))
M = cp.Variable(X_basis[0].shape)
N = cp.Variable(X_basis[0].shape)

constraints = [N >> 0]
constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
constraints += [N[0,0] == 1 - eta]

for i in range(1,len(X)):
    constraints += [
        eta*X[i,i] + N[i,i] == M[i,i]
    ]

constraints += [M >> 0]
constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
constraints += [M[0,0] == 1]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)
prob.solve(solver=cp.MOSEK, verbose=False)

0.8417785146809114

In [117]:
eta = cp.Variable((1, 1))
alpha = cp.Variable((len(X_basis), 1))
beta = cp.Variable((len(X_basis), 1))
# M = cp.Variable(X_basis[0].shape)
# N = cp.Variable(X_basis[0].shape)

constraints = [sum([beta[j]*X_basis[j] for j in range(len(X_basis))]) >> 0]
# constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
constraints += [sum([beta[j]*X_basis[j] for j in range(len(X_basis))])[0,0] == 1 - eta]

for i in range(1,len(X)):
    constraints += [
        eta*X[i,i] + sum([beta[j]*X_basis[j] for j in range(len(X_basis))])[i,i] == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])[i,i]
    ]

constraints += [sum([alpha[j]*X_basis[j] for j in range(len(X_basis))]) >> 0]
# constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
constraints += [sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])[0,0] == 1]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)
prob.solve(solver=cp.MOSEK, verbose=False)

0.8417784938509926

In [118]:
constraints[-1].dual_variables

[Variable(())]

In [126]:
def braket(b,k,shape):
    BK = np.zeros(shape)
    BK[b,k] = 1
    return BK

In [128]:
x = cp.Variable((1, 1))
r = cp.Variable((1, 1))
gamma = cp.Variable((len(X)-1, 1))
B = cp.Variable(X_basis[0].shape)
A = cp.Variable(X_basis[0].shape)

constraints = [r == 1 + sum([gamma[i-1]*X[i,i] for i in range(1,len(X))])]
constraints += [sum([braket(i,i,X_basis[0].shape)*gamma[i-1] for i in range(1,len(X))]) >> A - x*braket(0,0,X_basis[0].shape)] 
constraints += [sum([braket(i,i,X_basis[0].shape)*gamma[i-1] for i in range(1,len(X))]) << -B + r*braket(0,0,X_basis[0].shape)] 

for i in range(len(X_basis)):
    constraints += [
        cp.trace(X_basis[i] @ A) == 0
    ]
    constraints += [
        cp.trace(X_basis[i] @ B) == 0
    ]
    
prob = cp.Problem(cp.Minimize(x+r),
                  constraints)

In [129]:
prob.solve(solver=cp.MOSEK, verbose=False)

0.8417785754109901

In [130]:
x.value

array([[0.47906978]])

In [132]:
x.value+r.value

array([[0.84177858]])